In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
from transformers import pipeline


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'datasets'

In [10]:
!pip install py7zr
!pip install datasets

     ---------------------------------------- 0.0/67.6 kB ? eta -:--:--
     ------------------ --------------------- 30.7/67.6 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 67.6/67.6 kB 909.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/357.3 kB ? eta -:--:--
     ------ -------------------------------- 61.4/357.3 kB 3.2 MB/s eta 0:00:01
     ----------- -------------------------- 112.6/357.3 kB 1.3 MB/s eta 0:00:01
     ------------------------- ------------ 235.5/357.3 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 357.3/357.3 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
     ---------------------------------------- 46.1/46.1 kB 2.2 MB/s eta 0:00:00
  Using cached psutil-5.9.8-cp37-abi3-win_amd64.whl (255 kB)
     ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.8 MB 4.8 MB/s eta 0:00:01
     ---- ---


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
train_file = 'train.csv'
test_file = 'test.csv'
validation_file = 'validate.csv'

dataset_config = {
    'train': train_file,
    'test': test_file,
    'validation': validation_file
}

foren_dataset = load_dataset('csv', data_files=dataset_config)

for split in foren_dataset.keys():
    print(f"{split} set - Num rows: {len(foren_dataset[split])}")

train_dataset = foren_dataset['train']
test_dataset = foren_dataset['test']
validation_dataset = foren_dataset['validation']

In [ ]:
split_lengths = [len(foren_dataset[split]) for split in foren_dataset]
print(f"{split_lengths}")
print(f"{foren_dataset['train'].column_names}")
print("\nDialogue:")
print(foren_dataset["test"][0]["dialogue"])
print("\nSummary:")
print(foren_dataset["test"][0]["summary"])

In [ ]:
pipe = pipeline("summarization", model="kabita-choudhary/finetuned-bart-for-conversation-summary")
pipe_out = pipe(foren_dataset["test"][0]["dialogue"],min_length=5, max_length=20)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "kabita-choudhary/finetuned-bart-for-conversation-summary"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
d_len = [len(tokenizer.encode(s)) for s in foren_dataset["train"]["dialogue"]]
s_len = [len(tokenizer.encode(s)) for s in foren_dataset["train"]["summary"]]

In [ ]:
tokenizer.as_target_tokenizer()

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=256, truncation=True)
        
    return {
        "input_ids": input_encodings["input_ids"], 
        "attention_mask": input_encodings["attention_mask"],  
        "labels": target_encodings["input_ids"]
    }

In [ ]:
foren_dataset_p = foren_dataset.map(convert_examples_to_features, batched=True)

In [ ]:
columns = ["input_ids", "labels", "attention_mask"]
foren_dataset_p.set_format(type="torch", columns=columns)

In [ ]:
!pip install datasets
!pip install nltk
!pip install torch
!pip install rouge_score

In [ ]:
import torch
from datasets import load_metric
from nltk.translate.bleu_score import corpus_bleu

rouge_metric = load_metric("rouge")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_summaries = []
all_targets = []

for example in foren_dataset["test"]:
    inputs = tokenizer(example["dialogue"], return_tensors="pt", max_length=1024, truncation=True, padding="max_length").to(device)
    with torch.no_grad():
        summary_ids = model.generate(**inputs)
        summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    assert len(summaries) == 1, "More than one summary generated for an example"
    
    all_summaries.extend(summaries)
    all_targets.append(example["summary"])  

print("Length of all_summaries:", len(all_summaries))
print("Length of all_targets:", len(all_targets))

assert len(all_summaries) == len(all_targets), "Mismatch in the number of predictions and references"


rouge_result = rouge_metric.compute(predictions=all_summaries, references=all_targets)
print("ROUGE Scores:", rouge_result)

bleu_score = corpus_bleu([[ref] for ref in all_targets], all_summaries)
print("BLEU Score:", bleu_score)


In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments( output_dir="bart-large-cnn-samsum",
                                 num_train_epochs=3,
                                 warmup_steps=500,
                                 per_device_train_batch_size=1,
                                 per_gpu_eval_batch_size=1,
                                 weight_decay=0.01,
                                 logging_steps=10,
                                 evaluation_strategy='steps', 
                                 eval_steps=500,
                                 save_steps=1e6,
                                 gradient_accumulation_steps=16)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(model=model, 
                 args=training_args,
                 tokenizer=tokenizer,
                 data_collator=seq2seq_data_collator,
                 train_dataset=foren_dataset_p["train"],
                 eval_dataset=foren_dataset_p["validation"])

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("foren_summer_bart_large")
tokenizer.save_pretrained("foren_summer_bart_large")

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_metric
from nltk.translate.bleu_score import corpus_bleu

model_path = "/kaggle/working/foren_summer_bart_large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

rouge_metric = load_metric("rouge")

all_summaries = []
all_targets = []

for example in foren_dataset["test"]:
    inputs = tokenizer(example["dialogue"], return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    with torch.no_grad():
        summary_ids = model.generate(**inputs)
        summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    assert len(summaries) == 1, "More than one summary generated for an example"
    
    all_summaries.extend(summaries)
    all_targets.append(example["summary"])  

print("Length of all_summaries:", len(all_summaries))
print("Length of all_targets:", len(all_targets))

assert len(all_summaries) == len(all_targets), "Mismatch in the number of predictions and references"

rouge_result = rouge_metric.compute(predictions=all_summaries, references=all_targets)
print("ROUGE Scores:", rouge_result)

bleu_score = corpus_bleu([[ref] for ref in all_targets], all_summaries)
print("BLEU Score:", bleu_score)
